In [ ]:
"""
Created on Sun Oct 21 2018
@author: Kimin Lee
"""
import import_ipynb
import torch
import numpy as np
import calculate_log as callog
import models
import os
import lib_generation

from torchvision import transforms
from torch.autograd import Variable

import dataloader
import random

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [ ]:
def mahalanobis(model=None, trainloader=None, id_testloader=None, out_testloader=None, num_classes=10, magnitude=0.0014, std=(0,0,0), file_path=''):    
    # set information about feature extaction
    model.eval()
    temp_x = torch.rand(2,3,32,32).to(device)
    temp_x = Variable(temp_x)
    temp_list = model.feature_list(temp_x)[1]
    num_output = len(temp_list)
    feature_list = np.empty(num_output)
    count = 0
    for out in temp_list:
        feature_list[count] = out.size(1)
        count += 1
        
    print('get sample mean and covariance')
    sample_mean, precision = lib_generation.sample_estimator(model, num_classes, feature_list, trainloader)
    
    print('get Mahalanobis scores')
    for i in range(num_output):
        M_in = lib_generation.get_Mahalanobis_score(model, id_testloader, num_classes, file_path, \
                                                    True, std, sample_mean, precision, i, magnitude)
        M_in = np.asarray(M_in, dtype=np.float32)
        if i == 0:
            Mahalanobis_in = M_in.reshape((M_in.shape[0], -1))
        else:
            Mahalanobis_in = np.concatenate((Mahalanobis_in, M_in.reshape((M_in.shape[0], -1))), axis=1)
             
    for i in range(num_output):
        M_out = lib_generation.get_Mahalanobis_score(model, out_testloader, num_classes, file_path, \
                                                     False, std, sample_mean, precision, i, magnitude)
        M_out = np.asarray(M_out, dtype=np.float32)
        if i == 0:
            Mahalanobis_out = M_out.reshape((M_out.shape[0], -1))
        else:
            Mahalanobis_out = np.concatenate((Mahalanobis_out, M_out.reshape((M_out.shape[0], -1))), axis=1)
                
    Mahalanobis_in = np.asarray(Mahalanobis_in, dtype=np.float32)
    Mahalanobis_out = np.asarray(Mahalanobis_out, dtype=np.float32)
    Mahalanobis_data, Mahalanobis_labels = lib_generation.merge_and_generate_labels(Mahalanobis_out, Mahalanobis_in)
    
    if num_classes == 10:
        id_dataset='cifar10'
    else:
        id_dataset='cifar100'
        
    file_name = os.path.join(file_path, 'Mahalanobis_%s_%s_%s.npy' % (str(magnitude), id_dataset, 'svhn'))
    Mahalanobis_data = np.concatenate((Mahalanobis_data, Mahalanobis_labels), axis=1)
    np.save(file_name, Mahalanobis_data)